Вам предлагается предсказать рейтинг ресторана, используя датасет отзывов ресторанов

Цель этого курса - научить анализировать данные, а не строить ML модели. Поэтому в этой лабораторной мы предлагаем построить новые полезные для предсказания признаки, удалить бесполезные и обработать сырые.

Также разрешается использовать только алгоритм KNeighborsRegressor из python пакета sklearn. Использовать другие модели при решении нельзя. Разрешается и рекомендуется придумывать дополнительные признаки для датасета и находить наилучшие гиперпараметры алгоритма

In [1]:
import copy
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import sqrt
from numpy import log

from tqdm import tqdm

from sklearn.datasets import *
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df_train_init = pd.read_csv('data/andan-2022-autumn-lab-3/train.csv')
df_train_init.head()

,user_id,org_id,rating,ts,user_city,org_city,average_bill,rating_org,rubrics,food_delivery,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
0,12182789761588196310,11946401375069641937,4.0,677,msk,msk,1500.0,3.883212,Ресторан,1,...,0,0,0,0,0,0,0,0,0,0
1,2796544982129273287,5545859981678495568,5.0,577,msk,msk,1000.0,4.495379,Ресторан,1,...,0,0,0,0,0,0,0,0,0,0
2,10546824832088319728,3451485117888180206,4.0,716,msk,msk,500.0,4.576063,Кафе,1,...,0,0,0,0,0,0,0,0,0,0
3,9967742981458207920,9490041040735762889,3.0,639,spb,spb,1000.0,3.775701,Кафе,1,...,0,0,0,0,0,0,0,0,0,0
4,11046819495825038093,5234254278902729372,5.0,669,msk,msk,500.0,4.187500,Пиццерия,1,...,0,1,0,0,0,0,0,0,0,0


In [3]:
df_train_init.describe()

,user_id,org_id,rating,ts,average_bill,rating_org,food_delivery,breakfast,takeaway,summer_terrace,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
count,7.399500e+04,7.399500e+04,73995.000000,73995.000000,45433.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,...,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000,73995.000000
mean,9.196627e+18,9.038261e+18,4.322779,714.540766,1075.891973,4.319332,0.757119,0.576607,0.649206,0.602554,...,0.008014,0.007406,0.017677,0.002149,0.002419,0.001027,0.000932,0.001230,0.002960,0.001216
std,5.312861e+18,5.275833e+18,1.130370,293.492213,2733.507688,0.308584,0.428827,0.494100,0.477222,0.489373,...,0.089162,0.085739,0.131775,0.046306,0.049125,0.032032,0.030523,0.035047,0.054323,0.034854
min,1.445171e+14,1.626686e+15,1.000000,0.000000,500.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.614445e+18,4.504716e+18,4.000000,505.000000,500.000000,4.180952,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.191287e+18,9.104453e+18,5.000000,699.000000,500.000000,4.368304,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.376070e+19,1.350392e+19,5.000000,958.000000,1000.000000,4.524001,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.844666e+19,1.844531e+19,5.000000,1216.000000,246500.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df_train_init.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73995 entries, 0 to 73994
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         73995 non-null  uint64 
 1   org_id                          73995 non-null  uint64 
 2   rating                          73995 non-null  float64
 3   ts                              73995 non-null  int64  
 4   user_city                       73995 non-null  object 
 5   org_city                        73995 non-null  object 
 6   average_bill                    45433 non-null  float64
 7   rating_org                      73995 non-null  float64
 8   rubrics                         73995 non-null  object 
 9   food_delivery                   73995 non-null  int64  
 10  breakfast                       73995 non-null  int64  
 11  takeaway                        73995 non-null  int64  
 12  summer_terrace                  

In [5]:
df_test_init = pd.read_csv('data/andan-2022-autumn-lab-3/test_x.csv')
df_test_init.head()

,user_id,org_id,rating,ts,user_city,org_city,average_bill,rubrics,food_delivery,breakfast,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
0,7016090648418898986,17145242605301041786,5.0,754,spb,spb,1000.0,Ресторан,1,1,...,0,0,0,0,0,0,0,0,0,0
1,7984587338632361094,6994879043376924376,5.0,99,spb,spb,1500.0,Ресторан,1,1,...,0,0,0,0,0,0,0,0,0,0
2,12147560281443849509,5596242631024337097,5.0,1174,spb,spb,500.0,Кафе,1,0,...,0,0,0,0,0,0,0,0,0,0
3,9761706959109988045,16272893617101942587,5.0,1176,msk,msk,500.0,Кафе,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5657837819564452133,3193067678075550767,4.0,1030,msk,msk,NaN,Кафе,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
pd.read_csv('data/andan-2022-autumn-lab-3/test_x.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24665 entries, 0 to 24664
Data columns (total 71 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         24665 non-null  uint64 
 1   org_id                          24665 non-null  uint64 
 2   rating                          24665 non-null  float64
 3   ts                              24665 non-null  int64  
 4   user_city                       24665 non-null  object 
 5   org_city                        24665 non-null  object 
 6   average_bill                    15187 non-null  float64
 7   rubrics                         24665 non-null  object 
 8   food_delivery                   24665 non-null  int64  
 9   breakfast                       24665 non-null  int64  
 10  takeaway                        24665 non-null  int64  
 11  summer_terrace                  24665 non-null  int64  
 12  wi_fi                           

In [7]:
pd.read_csv('data/andan-2022-autumn-lab-3/test_x.csv').describe()

,user_id,org_id,rating,ts,average_bill,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
count,2.466500e+04,2.466500e+04,24665.000000,24665.000000,15187.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,...,24665.000000,24665.000000,24665.000000,24665.00000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000
mean,9.194880e+18,9.088017e+18,4.321589,716.566390,1105.517877,0.762660,0.580215,0.655423,0.604581,0.864545,...,0.007987,0.007622,0.018366,0.00223,0.002027,0.000973,0.001135,0.001460,0.002149,0.000973
std,5.324887e+18,5.261855e+18,1.128989,292.781347,3605.794698,0.425461,0.493534,0.475240,0.488950,0.342216,...,0.089014,0.086973,0.134274,0.04717,0.044979,0.031179,0.033674,0.038177,0.046306,0.031179
min,4.826883e+14,1.626686e+15,1.000000,0.000000,500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.567401e+18,4.667589e+18,4.000000,507.000000,500.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.235276e+18,9.104453e+18,5.000000,700.000000,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.379081e+19,1.354245e+19,5.000000,958.000000,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.844600e+19,1.844622e+19,5.000000,1216.000000,250000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df_train_init = df_train_init.drop(['user_id', 'org_id', 'user_city', 'org_city', 'rubrics'], axis=1)
df_train_init = df_train_init.replace(np.nan, 0)

df_train_init['ts'] = df_train_init['ts'].replace(0, 1)
df_train_init['average_bill'] = df_train_init['average_bill'].replace(0, 1)
df_train_init['money'] = (log(df_train_init['ts']) + log(df_train_init['average_bill'])) / 2
                         
df_train_init.head()

,rating,ts,average_bill,rating_org,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,business_lunch,...,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods,money
0,4.0,677,1500.0,3.883212,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,6.915446
1,5.0,577,1000.0,4.495379,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,6.632799
2,4.0,716,500.0,4.576063,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,6.394144
3,3.0,639,1000.0,3.775701,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,6.683830
4,5.0,669,500.0,4.187500,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,6.360196


In [9]:
df_test_init = df_test_init.drop(['user_id', 'org_id', 'user_city', 'org_city', 'rubrics'], axis=1)
df_test_init = df_test_init.replace(np.nan, 0)

df_test_init['ts'] = df_test_init['ts'].replace(0, 1)
df_test_init['average_bill'] = df_test_init['average_bill'].replace(0, 1)
df_test_init['money'] = (log(df_test_init['ts']) + log(df_test_init['average_bill'])) / 2

df_test_init.head()

,rating,ts,average_bill,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,business_lunch,payment_by_credit_card,...,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods,money
0,5.0,754,1000.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,6.766574
1,5.0,99,1500.0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,5.954170
2,5.0,1174,500.0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.641390
3,5.0,1176,500.0,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,6.642241
4,4.0,1030,1.0,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,3.468657


In [10]:
print(df_train_init['money'].min())
print(df_train_init['money'].max())

0.0
9.679119847642156


In [11]:
df_test_init.describe()

,rating,ts,average_bill,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,business_lunch,payment_by_credit_card,...,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods,money
count,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,...,24665.000000,24665.000000,24665.00000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000,24665.000000
mean,4.321589,716.566471,681.085668,0.762660,0.580215,0.655423,0.604581,0.864545,0.621893,0.903791,...,0.007622,0.018366,0.00223,0.002027,0.000973,0.001135,0.001460,0.002149,0.000973,5.296640
std,1.128989,292.781149,2879.937596,0.425461,0.493534,0.475240,0.488950,0.342216,0.484924,0.294884,...,0.086973,0.134274,0.04717,0.044979,0.031179,0.033674,0.038177,0.046306,0.031179,1.656893
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,507.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.354042
50%,5.000000,700.000000,500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.275717
75%,5.000000,958.000000,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.644786
max,5.000000,1216.000000,250000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.741401


In [12]:
train_y = df_train_init['rating_org'].to_numpy()
df_train_init = df_train_init.drop(['rating_org'], axis=1)
train_x = df_train_init.to_numpy()

In [13]:
KNR = KNeighborsRegressor(n_neighbors=int(sqrt(len(train_x))), weights='distance')
KNR.fit(train_x, train_y)

KNeighborsRegressor(n_neighbors=272, weights='distance')

In [14]:
size = df_test_init.shape[0]
indices = []
predicted = []

In [15]:
for i in tqdm(range(0, size)):
    indices = np.append(indices, i)
    test = df_test_init.iloc[i].to_numpy().reshape(1, -1)
    res = KNR.predict(test)
    predicted = np.append(predicted, res)

100%|████████████████████████████████████████████████████████████████████████████| 24665/24665 [08:44<00:00, 47.03it/s]


In [16]:
df_res = pd.DataFrame({'id': indices.astype(int), 'rating_org': predicted})
df_res.head()

,id,rating_org
0,0,4.333009
1,1,4.297331
2,2,4.288368
3,3,4.307777
4,4,4.334758


In [17]:
df_res.to_csv('sample_submission.csv', index=False)